## AIHub Json Parsing

### Development Environment

In [ ]:
%pip install kss==3.7.3

KSS Argument Error: Restart Jupyter Kernel Runtime

In [ ]:
%pip install python-mecab-ko

KSS 3.7.3 matches python-mecab-ko

In [ ]:
%pip install pandas

In [1]:
import re
import os
import kss
import json
from mecab import MeCab
import pandas as pd
from glob import glob

In [2]:
pwd

'c:\\Users\\MinSeok\\Documents\\AIHUB'

### Function

In [3]:
def json_file_name_list(path_list):
    for i in path_list:
        file_name = glob(i, recursive = True)
    return file_name

In [4]:
def train_valid_json_file_name_list(path_list):
    for i in path_list:
        if 'rain' in i:
            train_file_name = glob(i, recursive = True)
        elif 'alid' in i:  
            valid_file_name = glob(i, recursive = True)
    return train_file_name, valid_file_name

In [5]:
def divide_source_file_list(l, n): 
    for i in range(0, len(l), n): 
        yield l[i:i + n] 

In [6]:
def txt_file_name_list(source_file_nested_list, folder_corpus_type_name):
   
  text_file_name_list = []

  for i in range(len(source_file_nested_list)):
    txt_file_name = folder_corpus_type_name + str(i) + ".txt"
    text_file_name_list.append(txt_file_name)

  return text_file_name_list

In [7]:
def train_valid_txt_file_name_list(source_file_nested_list, folder_corpus_type_name):
   
  train_valid_set = source_file_nested_list[0][0]
  text_file_name_list = []

  if 'rain' in train_valid_set:
    for i in range(len(source_file_nested_list)):
      txt_file_name = folder_corpus_type_name + str(i) + ".txt"
      text_file_name_list.append(txt_file_name)

  elif 'alid' in train_valid_set:
    for i in range(len(source_file_nested_list)):
      txt_file_name = folder_corpus_type_name + str(i) + ".txt"
      text_file_name_list.append(txt_file_name)  

  return text_file_name_list

In [8]:
def formal_preprocessing_text(source):
    preprocessing_sentence_list = []
    
    source = source.strip()
    # strip으로 앞뒤 공백 제거
    for sentence in kss.split_sentences(source, use_heuristic=False,
                                        num_workers=32):
    # Formal articles (wiki, news, essays): recommend to False

        if re.search("^[A-Za-z0-9ㄱ-ㅎ가-힣一-鿕㐀-䶵豈-龎]", sentence[0]) is not None and \
            bool(re.match(r'[.]|[!]|[?]', sentence[-1])) == True and \
            len(sentence.split()) > 5:
            sentence = re.sub(r"\[.*?\]|\{.*?\}", "",  sentence)
            # The String starts with a letter
            # The String ends with [. ! ?]
            # 기타 괄호 제거할 시 괄호 내부에 모든 텍스트 제거

            sentence = re.sub(r"[^A-Za-z0-9ㄱ-ㅎ가-힣一-鿕㐀-䶵豈-龎()+-]", " ", sentence)
            # 특수문자 제거(영어 대소문자, 한글, 한자, 숫자, -, +, 소괄호 제외)

            sentence = sentence.strip()
            # strip으로 앞뒤 공백 제거
            
            total_length = len(sentence.replace(" " , ""))
            hangeul_length = len(re.sub(r"[^ㄱ-ㅣ가-힣\s]", "", sentence))
            hangeul_ratio = hangeul_length / total_length
            if hangeul_ratio >= 0.5:
            # 한글이 아닌 문자열이 50% 이상이 넘은 문장 제외
                preprocessing_sentence_list.append(sentence)

    return preprocessing_sentence_list

In [8]:
def informal_preprocessing_text(source):
    preprocessing_sentence_list = []
    
    source = source.strip()
    # strip으로 앞뒤 공백 제거
    for sentence in kss.split_sentences(source, use_heuristic=True,
                                        num_workers=32):
    # Formal articles (wiki, news, essays): recommend to False

        if re.search("^[A-Za-z0-9ㄱ-ㅎ가-힣一-鿕㐀-䶵豈-龎]", sentence[0]) is not None and \
            bool(re.match(r'[.]|[!]|[?]', sentence[-1])) == True and \
            len(sentence.split()) > 5:
            sentence = re.sub(r"\[.*?\]|\{.*?\}", "",  sentence)
            # The String starts with a letter
            # The String ends with [. ! ?]
            # 기타 괄호 제거할 시 괄호 내부에 모든 텍스트 제거

            sentence = re.sub(r"[^A-Za-z0-9ㄱ-ㅎ가-힣一-鿕㐀-䶵豈-龎()+-]", " ", sentence)
            # 특수문자 제거(영어 대소문자, 한글, 한자, 숫자, -, +, 소괄호 제외)

            sentence = sentence.strip()
            # strip으로 앞뒤 공백 제거
            
            total_length = len(sentence.replace(" " , ""))
            hangeul_length = len(re.sub(r"[^ㄱ-ㅣ가-힣\s]", "", sentence))
            hangeul_ratio = hangeul_length / total_length
            if hangeul_ratio >= 0.5:
            # 한글이 아닌 문자열이 50% 이상이 넘은 문장 제외
                preprocessing_sentence_list.append(sentence)

    return preprocessing_sentence_list

### AIHUB 문서요약 텍스트

[Source](https://aihub.or.kr/aihubdata/data/view.do?currMenu=115&topMenu=100&aihubDataSe=realm&dataSetSn=97)

In [9]:
path_list = ['AIHUB_문서요약 텍스트/Training/'+ '/**/*.json', 
             'AIHUB_문서요약 텍스트/Validation/'+ '/**/*.json']
train_file_name, valid_file_name = train_valid_json_file_name_list(path_list)

In [18]:
the_number_of_file = len(train_file_name) + len(valid_file_name)
print("The number of file:", the_number_of_file)
n = 1
print("The number of list element:", n)
train_file_nested_list = list(divide_source_file_list(train_file_name, n))
valid_file_nested_list = list(divide_source_file_list(valid_file_name, n))

The number of file: 6
The number of list element: 1


In [21]:
train_text_file_name_list = train_valid_txt_file_name_list(train_file_nested_list,
                                                           "demo/document_summary_text/AIHUB_document_summary_text_train_")
valid_text_file_name_list = train_valid_txt_file_name_list(valid_file_nested_list,
                                                           "demo/document_summary_text/AIHUB_document_summary_text_valid_")
the_numer_of_txt_file = len(train_text_file_name_list) + len(valid_text_file_name_list)
print("The number of txt file:", the_numer_of_txt_file)

The number of txt file: 6


In [50]:
def list_length_checker(source_file_nested_list, batch_size):
    
    the_number_of_total_txt_file = 0
    
    temp_nested_index = []
    
    for source_file_list in source_file_nested_list:   
        temp_index = []
        for i in range(len(source_file_list)):
            sentence_list = []
                
            if type(source_file_list) == str:
                source_file = source_file_list

            elif type(source_file_list) != str:
                source_file = source_file_list[i]
            

            with open(source_file, 'r', encoding='utf-8') as one_json_file:
                one_json_sample = json.load(one_json_file)
                
            source_df = pd.DataFrame(one_json_sample['documents'])
            source_dict = dict(source_df['text'].explode())
            source_json = pd.json_normalize(source_dict)  
            
            source_list = []
            for source_index in source_json:
                for source_nested_list in source_json[source_index]:
                    for source_single_list in source_nested_list:
                        source_sentence = ""
                        for source_single in source_single_list:
                            if type(source_single) == dict:
                                for key, value in source_single.items():
                                    if key == "sentence":
                                        source_sentence += " " + value
                        if len(source_sentence) > 0:
                            source_list.append(source_sentence)

            if len(source_list) >= 1000:
                the_number_of_txt_file = ((len(source_list) // batch_size) + 1)
                print("File:", source_file)    
                print("Index:", i, "  ", "Length of Source List:", len(source_list), \
                "  ", "The number of txt file:", the_number_of_txt_file, "\n")
                the_number_of_total_txt_file +=  the_number_of_txt_file
            else:
                the_number_of_total_txt_file += 1
                if i > 0 and the_number_of_file % i == 0:
                    temp_index.append(i)
                    try:
                        if i == temp_nested_index[0][0] and len(temp_nested_index) <= 1:
                            print("[For Example]")
                            print("This is not subject of batch. It's small source list")                            
                            print("File:", source_file)
                            print("Length of Source List:", len(source_list), "\n")
                    except:
                        pass
                    
        temp_nested_index.append(temp_index)
    
    print("")            
    print("Batch Size:", batch_size)
    print("The number of txt file:", the_number_of_total_txt_file)
    
    return the_number_of_total_txt_file

In [51]:
def make_corpus_txt_with_batch_list(source_file_nested_list,
                                    text_file_name_list,
                                    batch_size, the_number_of_txt_file):
    
  print("[Size]")
  print("The numnber of preprocessing corpus: " + str(the_number_of_txt_file))
  print("\n[Order]")
  num = 0
  for i in range(len(source_file_nested_list)):
    source_file_list = source_file_nested_list[i]
    
    for j in range(len(source_file_list)):

      if type(source_file_list) == str:
        source_file = source_file_list

      elif type(source_file_list) != str:
        source_file = source_file_list[j]

      with open(source_file, 'r', encoding='utf-8') as one_json_file:
        one_json_sample = json.load(one_json_file)

      source_df = pd.DataFrame(one_json_sample['documents'])
      source_dict = dict(source_df['text'].explode())
      source_json = pd.json_normalize(source_dict)  
      
      source_list = []
      for source_index in source_json:
          for source_nested_list in source_json[source_index]:
              for source_single_list in source_nested_list:
                  source_sentence = ""
                  for source_single in source_single_list:
                      if type(source_single) == dict:
                          for key, value in source_single.items():
                              if key == "sentence":
                                  source_sentence += " " + value
                  if len(source_sentence) > 0:
                    source_list.append(source_sentence)
      
      n = batch_size
      source_batch_list = list(divide_source_file_list(source_list, n))
      
      for source_list in source_batch_list:
        sentence_list = []                    
        for source in source_list:
            sentences = formal_preprocessing_text(source)
            for sentence in sentences:
              sentence_list.append(sentence)   

        num += 1
        print(str(num), end=" ")  
        
        with open(os.path.join('AIHUB_corpus/' + text_file_name_list[i][:-4] + "_" + str(num) + ".txt"), "a", encoding='utf-8') as fp:        
            fp.write("\n".join(sentence_list))              

In [52]:
batch_size = 1000
the_number_of_train_txt_file = list_length_checker(train_file_nested_list, batch_size)

File: AIHUB_문서요약 텍스트/Training\법률_train_original\train_original.json
Index: 0    Length of Source List: 42178    The number of txt file: 43 

File: AIHUB_문서요약 텍스트/Training\사설_train_original\train_original.json
Index: 0    Length of Source List: 242550    The number of txt file: 243 

File: AIHUB_문서요약 텍스트/Training\신문기사_train_original\train_original.json
Index: 0    Length of Source List: 2187380    The number of txt file: 2188 


Batch Size: 1000
The number of txt file: 2474


In [53]:
batch_size = 1000
the_number_of_valid_txt_file = list_length_checker(valid_file_nested_list, batch_size)

File: AIHUB_문서요약 텍스트/Validation\법률_valid_original\valid_original.json
Index: 0    Length of Source List: 4780    The number of txt file: 5 

File: AIHUB_문서요약 텍스트/Validation\사설_valid_original\valid_original.json
Index: 0    Length of Source List: 25202    The number of txt file: 26 

File: AIHUB_문서요약 텍스트/Validation\신문기사_valid_original\valid_original.json
Index: 0    Length of Source List: 239892    The number of txt file: 240 


Batch Size: 1000
The number of txt file: 271


In [ ]:
batch_size = 1000
make_corpus_txt_with_batch_list(train_file_nested_list, train_text_file_name_list,
                batch_size, the_number_of_train_txt_file)

In [ ]:
batch_size = 1000
make_corpus_txt_with_batch_list(valid_file_nested_list, valid_text_file_name_list,
                batch_size, the_number_of_valid_txt_file)

In [ ]:
corpus_list = glob("AIHUB_corpus/demo/document_summary_text/AIHUB_document_summary_text_train_" +"*.txt")
corpus_list += glob("AIHUB_corpus/demo/document_summary_text/AIHUB_document_summary_text_valid_" +"*.txt")

In [ ]:
with open('AIHUB_corpus/demo/AIHUB_document_summary_text.txt', 'w') as f:
    for corpus in corpus_list:
        with open(corpus) as text:
            for line in text:
                f.write(line)